In [1]:
from matplotlib import pyplot as plt
import h5py
import numpy as np
import matplotlib.gridspec as gridspec
from scipy import spatial
from tqdm import *
import random
import os
import subprocess

cdm = h5py.File('Data/COLOR_CDM_DM_subHaloes_z0.00_v2.hdf5', 'r')
wdm = h5py.File('Data/COLOR_WDM_DM_subHaloes_z0.00_v2.hdf5', 'r')

cdm_galaxy = h5py.File('Data\COLOR_CDM_galaxies_z0.00.hdf5', 'r')
wdm_galaxy = h5py.File('Data\COLOR_WDM_galaxies_z0.00.hdf5', 'r')

print(cdm,wdm)

<HDF5 file "COLOR_CDM_DM_subHaloes_z0.00_v2.hdf5" (mode r)> <HDF5 file "COLOR_WDM_DM_subHaloes_z0.00_v2.hdf5" (mode r)>


# Find all galaxies that are as heavy as the Milky Way
### Access the masses and positions of CDM and WDM galaxies

In [2]:
# Access mass/position for wdm/cdm
print(list(cdm_galaxy.keys()),list(wdm_galaxy.keys()))

cdm_galaxy_color = cdm_galaxy['Colour'][:]
cdm_galaxy_position = cdm_galaxy['GalaxyPos'][:]
cdm_galaxy_mass = cdm_galaxy['StellarMass'][:]

wdm_galaxy_color = wdm_galaxy['Colour'][:]
wdm_galaxy_position = wdm_galaxy['GalaxyPos'][:]
wdm_galaxy_mass = wdm_galaxy['StellarMass'][:]

cdm_galaxy_color, cdm_galaxy_position, cdm_galaxy_mass, wdm_galaxy_color, wdm_galaxy_position, wdm_galaxy_mass

['Colour', 'GalaxyPos', 'HaloRvir', 'HostHaloMass', 'IsCentral', 'StellarMass'] ['Colour', 'GalaxyPos', 'HaloRvir', 'HostHaloMass', 'IsCentral', 'StellarMass']


(array([0.3074894 , 0.59537411, 0.59701824, ..., 0.59878778, 0.59881878,
        0.5967443 ]),
 array([[17.43208122, 80.25539398, 54.00876999],
        [17.43210793, 80.22522736, 53.97425079],
        [17.43374443, 80.27178955, 54.02692413],
        ...,
        [85.55805969,  1.73876286, 65.28675079],
        [52.84350586, 10.11826706, 16.42269516],
        [55.3265419 , 34.21372986, 28.35575867]]),
 array([1.86803984e+08, 3.18429395e+04, 4.78547754e+03, ...,
        5.03134871e+00, 2.88395190e+00, 2.04510657e+03]),
 array([0.29522324, 0.29758644, 0.5930109 , ..., 0.59763646, 0.59827816,
        0.59777474]),
 array([[17.61932182, 79.06235504, 52.68593979],
        [18.65647697, 80.58162689, 52.30108261],
        [18.58250046, 80.91363525, 52.45199203],
        ...,
        [78.62889862, 83.76999664, 61.69128036],
        [97.51596069, 36.17467499, 37.83997726],
        [ 6.65006733, 48.75699997, 51.62447739]]),
 array([3.12070700e+06, 4.30492600e+06, 8.09484188e+05, ...,
        1.59

### Find all central galaxies within the mass range $2-6*10^{10}$ M$_{\odot}$

In [3]:
# get the indices
cdm_MW_galaxy_ind = np.where((cdm_galaxy_mass>2*10**10) & (cdm_galaxy_mass<6*10**10) & (cdm_galaxy['IsCentral'][:]==1))[0]
wdm_MW_galaxy_ind = np.where((wdm_galaxy_mass>2*10**10) & (wdm_galaxy_mass<6*10**10) & (wdm_galaxy['IsCentral'][:]==1))[0]

In [4]:
# get the positions
cdm_MW_galaxy_position = cdm_galaxy_position[cdm_MW_galaxy_ind]
wdm_MW_galaxy_position = wdm_galaxy_position[wdm_MW_galaxy_ind]

### Set a minimum mass of galaxies to search through

In [5]:
# get the indices
m_min=0
cdm_galaxy_filtered_ind = np.where(cdm_galaxy_mass > m_min)[0]
wdm_galaxy_filtered_ind = np.where(wdm_galaxy_mass > m_min)[0]

In [6]:
# get the positions
cdm_galaxy_position_filtered = cdm_galaxy_position[cdm_galaxy_filtered_ind]
wdm_galaxy_position_filtered = wdm_galaxy_position[wdm_galaxy_filtered_ind]

### Find all the galaxies within 3 Mpc of each MW-sized galaxy

In [7]:
# create the trees
cdm_tree = spatial.cKDTree(cdm_galaxy_position_filtered)
wdm_tree = spatial.cKDTree(wdm_galaxy_position_filtered)

In [8]:
# create an empty array for the indices of the galaxies
cdm_3Mpc_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_galaxy_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = cdm_tree.query_ball_point(cdm_MW_galaxy_position[i], 3)
    
    # record the indices for each MW-sized galaxy
    cdm_3Mpc_sphere_galaxy.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 456/456 [00:00<00:00, 3620.52it/s]


In [9]:
# create an empty array for the indices of the galaxies
wdm_3Mpc_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_galaxy_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = wdm_tree.query_ball_point(wdm_MW_galaxy_position[i], 3)
    
    # record the indices for each MW-sized galaxy
    wdm_3Mpc_sphere_galaxy.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 407/407 [00:00<00:00, 5090.13it/s]


### Find all the galaxies within the Virial Radius of each MW-sized galaxy

In [10]:
# create an empty array for the indices of the galaxies
cdm_VR_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_galaxy_position)):
    
    # find the indices of all galaxies within the virial radius of each MW-sized galaxy's host halo
    ind = cdm_tree.query_ball_point(cdm_MW_galaxy_position[i], cdm_galaxy['HaloRvir'][:][cdm_MW_galaxy_ind[i]])
    
    # record the indices for each MW-sized galaxy
    cdm_VR_sphere_galaxy.append(ind)

100%|███████████████████████████████████████████████████████████████████████████████| 456/456 [00:04<00:00, 110.33it/s]


In [11]:
# create an empty array for the indices of the galaxies
wdm_VR_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_galaxy_position)):
    
    # find the indices of all galaxies within the virial radius of each MW-sized galaxy's host halo
    ind = wdm_tree.query_ball_point(wdm_MW_galaxy_position[i], wdm_galaxy['HaloRvir'][:][wdm_MW_galaxy_ind[i]])
    
    # record the indices for each MW-sized galaxy
    wdm_VR_sphere_galaxy.append(ind)

100%|███████████████████████████████████████████████████████████████████████████████| 407/407 [00:01<00:00, 237.04it/s]


### Find the positions of galaxies in between the virial radius of each halo and 3 Mpc

In [12]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
cdm_shell_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_3Mpc_sphere_galaxy)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in cdm_3Mpc_sphere_galaxy[i] if j not in cdm_VR_sphere_galaxy[i]]
    
    # record the indices for each MW-sized galaxy
    cdm_shell_galaxy.append(cdm_galaxy_position[cdm_galaxy_filtered_ind][ind])

100%|████████████████████████████████████████████████████████████████████████████████| 456/456 [01:00<00:00,  7.57it/s]


In [13]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
wdm_shell_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_3Mpc_sphere_galaxy)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in wdm_3Mpc_sphere_galaxy[i] if j not in wdm_VR_sphere_galaxy[i]]
    
    # record the indices for each MW-sized galaxy
    wdm_shell_galaxy.append(wdm_galaxy_position[wdm_galaxy_filtered_ind][ind])

100%|████████████████████████████████████████████████████████████████████████████████| 407/407 [00:21<00:00, 19.07it/s]


# Find all halos that are as heavy as the Milky Way
### Access the masses and positions of CDM and WDM halos

In [14]:
# Access mass/position for wdm/cdm
print(list(cdm.keys()),list(wdm.keys()))

cdm_mass = cdm['SubhaloMass'][:]
cdm_position = cdm['SubhaloPos'][:]

wdm_mass = wdm['SubhaloMass'][:]
wdm_position = wdm['SubhaloPos'][:]

cdm_mass, cdm_position, wdm_mass, wdm_position

['IsCentral', 'R200', 'SubhaloMass', 'SubhaloPos'] ['IsCentral', 'R200', 'SubhaloMass', 'SubhaloPos']


(array([1.87617493e+14, 1.84468594e+14, 6.89978024e+13, ...,
        1.76011232e+08, 1.76011232e+08, 1.76011232e+08]),
 array([[10.93832207, 81.23078918, 54.77935028],
        [17.94562721, 79.86347961, 53.3843956 ],
        [15.62266159, 78.18474579, 52.83857346],
        ...,
        [ 1.47748184,  3.54455233, 99.53264618],
        [99.32190704,  2.56014371,  2.57368279],
        [99.0621109 ,  4.18381071,  2.89533401]]),
 array([1.89941540e+14, 1.86912732e+14, 6.83256610e+13, ...,
        1.76011232e+08, 1.76011232e+08, 1.76011232e+08]),
 array([[1.80150642e+01, 7.98655701e+01, 5.34328232e+01],
        [1.09048624e+01, 8.12610779e+01, 5.48179703e+01],
        [1.56323385e+01, 7.81665344e+01, 5.28157997e+01],
        ...,
        [6.02296066e+00, 5.05078554e+00, 9.79758301e+01],
        [5.75055540e-01, 3.26677370e+00, 3.52767438e-01],
        [2.07897183e-03, 3.23730040e+00, 9.97708917e-01]]))

### Find all central halos within the mass range $0.8-2*10^{12}$ M$_{\odot}$

In [15]:
# get the indices
cdm_MW_ind = np.where((cdm_mass>0.8*10**12) & (cdm_mass<2*10**12) & (cdm['IsCentral'][:]==1))[0]
wdm_MW_ind = np.where((wdm_mass>0.8*10**12) & (wdm_mass<2*10**12) & (wdm['IsCentral'][:]==1))[0]

len(cdm_MW_ind), len(wdm_MW_ind)

(1057, 1065)

In [16]:
# get the positions
cdm_MW_position = cdm_position[cdm_MW_ind]
wdm_MW_position = wdm_position[wdm_MW_ind]

### Find all the galaxies within 3 Mpc of each MW-sized halo

In [17]:
# create an empty array for the indices of the galaxies
cdm_3Mpc_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized halo
    ind = cdm_tree.query_ball_point(cdm_MW_position[i], 3)
    
    # record the indices for each MW-sized halo
    cdm_3Mpc_sphere.append(ind)

100%|████████████████████████████████████████████████████████████████████████████| 1057/1057 [00:00<00:00, 2781.53it/s]


In [18]:
# create an empty array for the indices of the galaxies
wdm_3Mpc_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized halo
    ind = wdm_tree.query_ball_point(wdm_MW_position[i], 3)
    
    # record the indices for each MW-sized halo
    wdm_3Mpc_sphere.append(ind)

100%|████████████████████████████████████████████████████████████████████████████| 1065/1065 [00:00<00:00, 4461.36it/s]


### Find all the galaxies within the Virial Radius of each MW-sized halo

In [19]:
# create an empty array for the indices of the galaxies
cdm_VR_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_position)):
    
    # find the indices of all galaxies within the virial radius of each MW-sized halo
    ind = cdm_tree.query_ball_point(cdm_MW_position[i], cdm['R200'][:][cdm_MW_ind[i]])
    
    # record the indices for each MW-sized halo
    cdm_VR_sphere.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 1057/1057 [00:40<00:00, 26.38it/s]


In [20]:
# create an empty array for the indices of the galaxies
wdm_VR_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_position)):
    
    # find the indices of all galaxies within the virial radius of each MW-sized halo
    ind = wdm_tree.query_ball_point(wdm_MW_position[i], wdm['R200'][:][wdm_MW_ind[i]])
    
    # record the indices for each MW-sized halo
    wdm_VR_sphere.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 1065/1065 [00:25<00:00, 41.63it/s]


### Find the positions of galaxies in between the virial radius of each halo and 3 Mpc

In [21]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
cdm_shell = []

# iterate through each MW-sized halo
for i in trange(len(cdm_3Mpc_sphere)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in cdm_3Mpc_sphere[i] if j not in cdm_VR_sphere[i]]
    
    # record the indices for each MW-sized halo
    cdm_shell.append(cdm_galaxy_position[cdm_galaxy_filtered_ind][ind])

100%|██████████████████████████████████████████████████████████████████████████████| 1057/1057 [01:25<00:00, 12.34it/s]


In [22]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
wdm_shell = []

# iterate through each MW-sized halo
for i in trange(len(wdm_3Mpc_sphere)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in wdm_3Mpc_sphere[i] if j not in wdm_VR_sphere[i]]
    
    # record the indices for each MW-sized halo
    wdm_shell.append(wdm_galaxy_position[wdm_galaxy_filtered_ind][ind])

100%|██████████████████████████████████████████████████████████████████████████████| 1065/1065 [00:38<00:00, 27.51it/s]


# Compute the average correlation function of MW-sized galaxies and halos

In [23]:
# Create an input folder 
home = "C:\\Users\\Mahlet Shiferaw\\OneDrive\\Dell\\Astro_98"
folder = home + "\\Input\\local\\"
if not os.path.exists(folder):
    os.mkdir(folder)
    
# iterate through each result
for position, label, name in zip([cdm_shell_galaxy, wdm_shell_galaxy, cdm_shell, wdm_shell], 
                                 ["cdm_", "wdm_", "cdm_", "wdm_"], ["local_galaxy_position", "local_galaxy_position",
                                                                    "local_halo_position", "local_halo_position"]):

    # iterate through each MW-sized cdm galaxy
    for i in range(len(position)):

        # save the galaxy positions
        file = label+name+str(i)+".txt"
        np.savetxt(folder + file, position[i], fmt = "%f %f %f")
        
        # zip each input file
        os.chdir(folder)
        subprocess.call(['zip', '-ur', name+'.zip', file])
        os.chdir(home)